##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Engenharia de características usando o TFX Pipeline e o TensorFlow Transform

***Transforme dados de entrada e treine um modelo com um pipeline TFX.***

Observação: recomendamos executar este tutorial em um notebook Colab, sem necessidade de configuração! Basta clicar em “Executar no Google Colab”.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tft"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/tfx/penguin_tft.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
<td>     <a target="_blank" href="https://github.com/tensorflow/tfx/tree/master/docs/tutorials/tfx/penguin_tft.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/tfx/docs/tutorials/tfx/penguin_tft.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

Neste tutorial baseado em notebook, criaremos e executaremos um pipeline TFX para ingerir dados de entrada brutos e pré-processá-los adequadamente para treinamento de ML. Este notebook é baseado no pipeline TFX que construímos em [Validação de dados usando TFX Pipeline e Tutorial TensorFlow Data Validation](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv). Se você ainda não leu esse documento, leia antes de prosseguir com este notebook.

Você pode aumentar a qualidade preditiva de seus dados e/ou reduzir a dimensionalidade com engenharia de características. Um dos benefícios de usar o TFX é que você escreverá seu código de transformação uma única vez, e as transformações resultantes serão consistentes entre o treinamento e serviço, a fim de evitar desvios de training/serving.

Adicionaremos um componente `Transform` ao pipeline. O componente Transform é implementado usando a biblioteca [tf.transform](https://www.tensorflow.org/tfx/transform/get_started).

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.

## Configuração

Primeiro precisamos instalar o pacote TFX Python e baixar o dataset que usaremos para nosso modelo.

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instale o TFX


In [ ]:
!pip install -U tfx

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

Verifique as versões do TensorFlow e TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuração de variáveis

Existem algumas variáveis ​​usadas para definir um pipeline. Você pode personalizar essas variáveis ​​como desejar. Por padrão, toda a saída do pipeline será gerada no diretório atual.

In [ ]:
import os

PIPELINE_NAME = "penguin-transform"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparação dos dados de exemplo

Faremos download do dataset de exemplo para uso no nosso pipeline TFX. O dataset que estamos usando é o [dataset Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html).

No entanto, ao contrário dos tutoriais anteriores que usaram um dataset já pré-processado, usaremos o dataset **bruto** do Palmer Penguins.


Como o componente TFX ExampleGen lê entradas de um diretório, precisamos criar um diretório e copiar o dataset para ele.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_path = 'https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_path, _data_filepath)

Dê uma olhada rápida na estrutura geral dos dados brutos.

In [ ]:
!head {_data_filepath}

Existem algumas entradas com valores ausentes que são representadas como `NA`. Vamos apenas excluir essas entradas neste tutorial.

In [ ]:
!sed -i '/\bNA\b/d' {_data_filepath}
!head {_data_filepath}

Você deverá ser capaz de ver sete características que descrevem os pinguins. Usaremos o mesmo conjunto de características dos tutoriais anteriores - 'culmen_length_mm', 'culmen_thought_mm', 'flipper_length_mm', 'body_mass_g' - e faremos a previsão da 'species' (espécie) de um pinguim.

**A única diferença será que os dados de entrada não são pré-processados.** Observe que não usaremos outras características como ‘island’ ou ‘sex’ neste tutorial.

### Prepare um arquivo de esquema

Conforme descrito no [Tutorial de validação de dados usando TFX Pipeline e TensorFlow Data Validation](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv), precisamos de um arquivo de esquema para o dataset. Como o dataset é diferente do tutorial anterior, precisaremos gerá-lo novamente. Neste tutorial, pularemos essas etapas e usaremos apenas um arquivo de esquema preparado.


In [ ]:
import shutil

SCHEMA_PATH = 'schema'

_schema_uri = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/schema/raw/schema.pbtxt'
_schema_filename = 'schema.pbtxt'
_schema_filepath = os.path.join(SCHEMA_PATH, _schema_filename)

os.makedirs(SCHEMA_PATH, exist_ok=True)
urllib.request.urlretrieve(_schema_uri, _schema_filepath)

Este arquivo de esquema foi criado com o mesmo pipeline do tutorial anterior, sem nenhuma alteração manual.

## Crie um pipeline

Os pipelines TFX são definidos usando APIs Python. Adicionaremos o componente `Transform` ao pipeline que criamos no [Tutorial de validação de dados](https://www.tensorflow.org/tfx/tutorials/tfx/penguin_tfdv).

Um componente Transform requer dados de entrada de um componente `ExampleGen` e um esquema de um componente `SchemaGen` e produz um "grafo de transformação". A saída será usada por um componente `Trainer`. A transformação pode opcionalmente produzir "dados transformados", que são os dados materializados após a transformação. No entanto, transformaremos os dados durante o treinamento neste tutorial sem materialização dos dados intermediários transformados.

É importante observar que precisamos definir uma função Python, `preprocessing_fn` para descrever como os dados de entrada devem ser transformados. Isso é semelhante a um componente Trainer que também requer código do usuário para definição do modelo.


### Escreva o código de pré-processamento e treinamento

Precisamos definir duas funções Python. Uma para o Transform e outra para o Trainer.

#### preprocessing_fn

O componente Transform encontrará uma função chamada `preprocessing_fn` no arquivo do módulo fornecido, como fizemos para o componente `Trainer`. Você também pode definir uma função específica usando o <a href="https://github.com/tensorflow/tfx/blob/142de6e887f26f4101ded7925f60d7d4fe9d42ed/tfx/components/transform/component.py#L113" data-md-type="link">parâmetro `preprocessing_fn`</a> do componente Transform.

Neste exemplo, faremos dois tipos de transformação. Para características numéricas contínuas como `culmen_length_mm` e `body_mass_g`, normalizaremos esses valores usando a função [tft.scale_to_z_score](https://www.tensorflow.org/tfx/transform/api_docs/python/tft/scale_to_z_score). Para as características de rótulo, precisamos converter rótulos de string em valores de índices numéricos. Usaremos [`tf.lookup.StaticHashTable`](https://www.tensorflow.org/api_docs/python/tf/lookup/StaticHashTable) para a conversão.

Para identificar facilmente os campos transformados, acrescentaremos um sufixo `_xf` aos nomes das características transformadas.

#### run_fn

O modelo em si é quase o mesmo dos tutoriais anteriores, mas desta vez transformaremos os dados de entrada usando o grafo de transformação do componente Transform.

Mais uma diferença importante em relação ao tutorial anterior é que agora exportamos um modelo para o serviço que inclui não apenas o grafo de computação do modelo, mas também o grafo de transformação para pré-processamento, que é gerado no componente Transform. Precisamos definir uma função separada que será usada para atender às solicitações recebidas. Você pode ver que a mesma função `_apply_preprocessing` foi usada para os dados de treinamento e para a solicitação de serviço.


In [ ]:
_module_file = 'penguin_utils.py'

In [ ]:
%%writefile {_module_file}


from typing import List, Text
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_metadata.proto.v0 import schema_pb2
import tensorflow_transform as tft
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

# Specify features that we will use.
_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10


# NEW: TFX Transform will call this function.
def preprocessing_fn(inputs):
  """tf.transform's callback function for preprocessing inputs.

  Args:
    inputs: map from feature keys to raw not-yet-transformed features.

  Returns:
    Map from string feature key to transformed feature.
  """
  outputs = {}

  # Uses features defined in _FEATURE_KEYS only.
  for key in _FEATURE_KEYS:
    # tft.scale_to_z_score computes the mean and variance of the given feature
    # and scales the output based on the result.
    outputs[key] = tft.scale_to_z_score(inputs[key])

  # For the label column we provide the mapping from string to index.
  # We could instead use `tft.compute_and_apply_vocabulary()` in order to
  # compute the vocabulary dynamically and perform a lookup.
  # Since in this example there are only 3 possible values, we use a hard-coded
  # table for simplicity.
  table_keys = ['Adelie', 'Chinstrap', 'Gentoo']
  initializer = tf.lookup.KeyValueTensorInitializer(
      keys=table_keys,
      values=tf.cast(tf.range(len(table_keys)), tf.int64),
      key_dtype=tf.string,
      value_dtype=tf.int64)
  table = tf.lookup.StaticHashTable(initializer, default_value=-1)
  outputs[_LABEL_KEY] = table.lookup(inputs[_LABEL_KEY])

  return outputs


# NEW: This function will apply the same transform operation to training data
#      and serving requests.
def _apply_preprocessing(raw_features, tft_layer):
  transformed_features = tft_layer(raw_features)
  if _LABEL_KEY in raw_features:
    transformed_label = transformed_features.pop(_LABEL_KEY)
    return transformed_features, transformed_label
  else:
    return transformed_features, None


# NEW: This function will create a handler function which gets a serialized
#      tf.example, preprocess and run an inference with it.
def _get_serve_tf_examples_fn(model, tf_transform_output):
  # We must save the tft_layer to the model to ensure its assets are kept and
  # tracked.
  model.tft_layer = tf_transform_output.transform_features_layer()

  @tf.function(input_signature=[
      tf.TensorSpec(shape=[None], dtype=tf.string, name='examples')
  ])
  def serve_tf_examples_fn(serialized_tf_examples):
    # Expected input is a string which is serialized tf.Example format.
    feature_spec = tf_transform_output.raw_feature_spec()
    # Because input schema includes unnecessary fields like 'species' and
    # 'island', we filter feature_spec to include required keys only.
    required_feature_spec = {
        k: v for k, v in feature_spec.items() if k in _FEATURE_KEYS
    }
    parsed_features = tf.io.parse_example(serialized_tf_examples,
                                          required_feature_spec)

    # Preprocess parsed input with transform operation defined in
    # preprocessing_fn().
    transformed_features, _ = _apply_preprocessing(parsed_features,
                                                   model.tft_layer)
    # Run inference with ML model.
    return model(transformed_features)

  return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for tuning/training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  dataset = data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(batch_size=batch_size),
      schema=tf_transform_output.raw_metadata.schema)

  transform_layer = tf_transform_output.transform_features_layer()
  def apply_transform(raw_features):
    return _apply_preprocessing(raw_features, transform_layer)

  return dataset.map(apply_transform).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [
      keras.layers.Input(shape=(1,), name=key)
      for key in _FEATURE_KEYS
  ]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """
  tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      tf_transform_output,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # NEW: Save a computation graph including transform layer.
  signatures = {
      'serving_default': _get_serve_tf_examples_fn(model, tf_transform_output),
  }
  model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Agora você concluiu todas as etapas de preparação para construir um pipeline TFX.

### Escreva uma definição de pipeline

Definimos uma função para criar um pipeline TFX. Um objeto `Pipeline` representa um pipeline do TFX que pode ser executado usando um dos sistemas de orquestração de pipeline suportados pelo TFX.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     schema_path: str, module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Implements the penguin pipeline with TFX."""
  # Brings data into the pipeline or otherwise joins/converts training data.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Computes statistics over data for visualization and example validation.
  statistics_gen = tfx.components.StatisticsGen(
      examples=example_gen.outputs['examples'])

  # Import the schema.
  schema_importer = tfx.dsl.Importer(
      source_uri=schema_path,
      artifact_type=tfx.types.standard_artifacts.Schema).with_id(
          'schema_importer')

  # Performs anomaly detection based on statistics and data schema.
  example_validator = tfx.components.ExampleValidator(
      statistics=statistics_gen.outputs['statistics'],
      schema=schema_importer.outputs['result'])

  # NEW: Transforms input data using preprocessing_fn in the 'module_file'.
  transform = tfx.components.Transform(
      examples=example_gen.outputs['examples'],
      schema=schema_importer.outputs['result'],
      materialize=False,
      module_file=module_file)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],

      # NEW: Pass transform_graph to the trainer.
      transform_graph=transform.outputs['transform_graph'],

      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  components = [
      example_gen,
      statistics_gen,
      schema_importer,
      example_validator,

      transform,  # NEW: Transform component was added to the pipeline.

      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## Execute o pipeline

Usaremos o `LocalDagRunner` como no tutorial anterior.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      schema_path=SCHEMA_PATH,
      module_file=_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Você deverá ver "INFO:absl:Component Pusher is finished." se o pipeline for concluído com sucesso.

O componente pusher envia o modelo treinado para `SERVING_MODEL_DIR` que é o diretório `serving_model/penguin-transform` se você não alterou as variáveis ​​nas etapas anteriores. Você pode ver o resultado no navegador de arquivos no painel esquerdo do Colab ou usando o seguinte comando:

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

Você também pode verificar a assinatura do modelo gerado usando a [ferramenta `saved_model_cli`](https://www.tensorflow.org/guide/saved_model#show_command).

In [ ]:
!saved_model_cli show --dir {SERVING_MODEL_DIR}/$(ls -1 {SERVING_MODEL_DIR} | sort -nr | head -1) --tag_set serve --signature_def serving_default

Como definimos `serving_default` com nossa própria função `serve_tf_examples_fn`, a assinatura mostra que ela recebe uma única string. Esta string é uma string serializada de tf.Examples e será processada com a função [tf.io.parse_example()](https://www.tensorflow.org/api_docs/python/tf/io/parse_example) conforme definimos anteriormente (leia mais sobre tf.Examples [aqui](https://www.tensorflow.org/tutorials/load_data/tfrecord)).

Podemos carregar o modelo exportado e tentar algumas inferências com alguns exemplos.

In [ ]:
# Find a model with the latest timestamp.
model_dirs = (item for item in os.scandir(SERVING_MODEL_DIR) if item.is_dir())
model_path = max(model_dirs, key=lambda i: int(i.name)).path

loaded_model = tf.keras.models.load_model(model_path)
inference_fn = loaded_model.signatures['serving_default']

In [ ]:
# Prepare an example and run inference.
features = {
  'culmen_length_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[49.9])),
  'culmen_depth_mm': tf.train.Feature(float_list=tf.train.FloatList(value=[16.1])),
  'flipper_length_mm': tf.train.Feature(int64_list=tf.train.Int64List(value=[213])),
  'body_mass_g': tf.train.Feature(int64_list=tf.train.Int64List(value=[5400])),
}
example_proto = tf.train.Example(features=tf.train.Features(feature=features))
examples = example_proto.SerializeToString()

result = inference_fn(examples=tf.constant([examples]))
print(result['output_0'].numpy())

Espera-se que o terceiro elemento, que corresponde às espécies ‘Gentoo’, seja o maior dentre os três.

## Próximos passos

Se você quiser saber mais sobre o componente Transform, consulte o [guia do componente Transform](https://www.tensorflow.org/tfx/guide/transform). Você pode encontrar mais recursos em https://www.tensorflow.org/tfx/tutorials.

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.
